In [2]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        z=[]
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            a=key
            if len(a)>2:
                z.append(a)
            if i > number:
                break
        x= set(i for i in z if not any(i in s for s in z if i != s))
        n=list(x)
        n = ','.join([i for i in n])
        return(n)
    
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [5]:
text='''The Goldman Sachs Group, Inc. is a leading global investment banking, securities and investment management firm that provides a wide range of financial services to a substantial and diversified client base that includes corporations, financial institutions, governments and individuals.'''



In [11]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

req = Request("https://www.bbc.com/news/world-asia-india-53304160", headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req).read()
soup = BeautifulSoup(html)

#kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()

#get text
text = soup.get_text()

#break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
#break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)
print(text)

India coronavirus: Life-saving Covid-19 drugs sold on Delhi black market - BBC News
HomepageAccessibility linksSkip to contentAccessibility Help BBC AccountNotifications HomeNewsSportWeatheriPlayerSoundsCBBCCBeebiesFoodBitesizeArtsTasterLocalThreeMenuSearchSearch the BBCSearch the BBC
News
BBC News Navigation
Sections
Home
Video
World
Asia
selected
UK
Business
Tech
Science
Stories
Entertainment & Arts
Health
World News TV
In Pictures
Reality Check
Newsbeat
Special Reports
Explainers
The Reporters
Have Your Say
Asia selected
China
India
selected
India
India
India coronavirus: Life-saving Covid-19 drugs sold on Delhi black market
7 July 2020
Share this with Facebook
Share this with Messenger
Share this with Twitter
Share this with Email
Share this with Facebook
Share this with WhatsApp
Share this with Messenger
Share this with Twitter
Share
Share this with
These are external links and will open in a new window
Email
Share this with Email
Facebook
Share this with Facebook
Messenger
Share 

In [6]:

#text= "The Goldman Sachs Group, Inc. is a leading global investment banking, securities and investment management firm that provides a wide range of financial services to a substantial and diversified client base that includes corporations, financial institutions, governments and individuals."
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(8)


'institutions,investment,base,range,Inc.,banking,services,client,corporations,securities'